In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PIL import ImageOps

class KittiPoint:
    def __init__(self,x,y):
        self.x = x
        self.y = y

class KittiFrame:
    
    def __init__(self,date,seriesId,frameId):
        self.date = date
        self.seriesId = seriesId
        self.frameId = frameId
        self.points = []

    def getFramePath(self):
        seriesIdLeadingZeros = '0000'[:-len(str(self.seriesId))]+str(self.seriesId)
        frameIdLeadingZeros = '0000000000'[:-len(str(self.frameId))]+str(self.frameId)
        return '../KITTI-dataset/kitti_base_dir/2011_%s/2011_%s_drive_%s_sync/image_02/data/%s.png'%(self.date,self.date,seriesIdLeadingZeros,frameIdLeadingZeros)

    def getImageId(self):
        return self.date+self.seriesId+self.frameId

    def addPoint(self,x,y):
        self.points.append(KittiPoint(x,y))
        
    def showImageWithPoints(self):
        plt.imshow(mpimg.imread(self.getFramePath()))
        for point in self.points:
            plt.scatter(point.x, point.y,s=10,c='r')
        plt.show()
        
    def getCentralPointYValueInArea(self,xStart,xEnd):
        pointsInArea = []
        for point in self.points:
            if(point.x >= xStart and point.x <= xEnd):
                pointsInArea.append(point)
        if len(pointsInArea) == 0:
            return -1
        # Sort by x poisition. It is not ensured that the values are sorted.
        pointsInArea.sort(key=lambda x: x.x)
        centralPoint = pointsInArea[round((len(pointsInArea)-1)/2)]
        return centralPoint.y

    def getImageStixel(self,offset,stixelWidth):
        image = Image.open(self.getFramePath())
        width = image.size[0]
        cropFromLeft = offset*stixelWidth 
        cropFromRight = width-(offset*stixelWidth)-stixelWidth
        if cropFromRight < 0:
            print('Offset too big for image.')
            return (None,-1)
        # left, up, right, bottom
        border = (cropFromLeft,0,cropFromRight, 0)
        return (ImageOps.crop(image, border),self.getCentralPointYValueInArea(offset*stixelWidth,(offset+1)*stixelWidth))
    
    def splitImageStixel(self,stixel,split_height):
        splits = []
        stixel_height = stixel.size[1]
        for i in range(0,stixel_height-split_height+1):
            cropped = ImageOps.crop(stixel, border)
            splits.append(cropped)
        return cropped

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PIL import ImageOps

class KittiPoint:
    def __init__(self,x,y):
        self.x = x
        self.y = y

class SidewalkFrame:
    
    def __init__(self,image_path):
        self.image_path = '../sidewalk-dataset/'+image_path
        self.points = []
        
    def addPoint(self,x,y):
        self.points.append(KittiPoint(x,y))
        
    def showImageWithPoints(self):
        plt.imshow(mpimg.imread(self.image_path))
        for point in self.points:
            plt.scatter(point.x, point.y,s=10,c='r')
        plt.show()
        
    def getCentralPointYValueInArea(self,xStart,xEnd):
        pointsInArea = []
        for point in self.points:
            if(point.x >= xStart and point.x <= xEnd):
                pointsInArea.append(point)
        if len(pointsInArea) == 0:
            return -1
        # Sort by x poisition. It is not ensured that the values are sorted.
        pointsInArea.sort(key=lambda x: x.x)
        centralPoint = pointsInArea[round((len(pointsInArea)-1)/2)]
        return centralPoint.y

    def getImageStixel(self,offset,stixelWidth):
        image = Image.open(self.image_path)
        width = image.size[0]
        cropFromLeft = offset*stixelWidth 
        cropFromRight = width-(offset*stixelWidth)-stixelWidth
        if cropFromRight < 0:
            print('Offset too big for image.')
            return (None,-1)
        # left, up, right, bottom
        border = (cropFromLeft,0,cropFromRight, 0)
        return (ImageOps.crop(image, border),self.getCentralPointYValueInArea(offset*stixelWidth,(offset+1)*stixelWidth))
    
    def splitImageStixel(self,stixel,split_height):
        splits = []
        stixel_height = stixel.size[1]
        for i in range(0,stixel_height-split_height+1):
            cropped = ImageOps.crop(stixel, border)
            splits.append(cropped)
        return cropped

In [ ]:
kittiFrames = {}
f = open("../KITTI-dataset/StixelsGroundTruth.txt","r")

lines = f.readlines()
unique_series = {}
for line in lines:
    line_s = line.rstrip().split("\t")
    kittiFrame = KittiFrame(line_s[0],line_s[1],line_s[2])
    if kittiFrames.get(kittiFrame.getImageId()) == None:
        kittiFrames[kittiFrame.getImageId()] = kittiFrame
    kittiFrame = kittiFrames[kittiFrame.getImageId()]
    kittiFrame.addPoint(int(line_s[3]),int(line_s[4]))

In [ ]:
from IPython.display import clear_output

sidewalkFrames = {}
f = open("../sidewalk-dataset/labels.csv","r")

lines = f.readlines()
unique_series = {}
for line in lines:
    line_s = line.rstrip().split(",")
    sidewalkFrame = SidewalkFrame(line_s[0])
    if sidewalkFrames.get(line_s[0]) == None:
        sidewalkFrames[line_s[0]] = sidewalkFrame
    sidewalkFrame = sidewalkFrames[line_s[0]]
    sidewalkFrame.addPoint(int(line_s[1]),int(line_s[2]))

In [ ]:
# Generate Stixels

import pandas as pd

STIXEL_WIDTH = 24

save_path = 'sidewalkStixels/'

rows_list = []
stixel_counter = 0
progress = 0
length = len(sidewalkFrames.keys())
for key in sidewalkFrames.keys():
    sidewalkFrame = sidewalkFrames.get(key)
    for stixel_number in range(0,27,1):
        stixel,y = sidewalkFrame.getImageStixel(stixel_number,STIXEL_WIDTH)
        path = '%s%i.png'%(save_path,stixel_counter)
        stixel.save(path)
        stixel_counter+=1
        row_dict = {}
        row_dict['img_path'] = path
        row_dict['y'] = y
        rows_list.append(row_dict)
    print((progress/length)*100)
    progress+=1
    clear_output(wait=True)
df = pd.DataFrame(rows_list)
df.to_csv(save_path+'../labels.csv')

In [ ]:
# Generate Stixels

save_path = 'data_train/stixels/'

rows_list = []
stixel_counter = 0
progress = 0
length = len(kittiFrames.keys())
for key in kittiFrames.keys():
    kittiFrame = kittiFrames.get(key)
    for stixel_number in range(0,maximum_offset,1):
        stixel,y = kittiFrame.getImageStixel(stixel_number,STIXEL_WIDTH)
        path = '%s%i.png'%(save_path,stixel_counter)
        stixel.save(path)
        stixel_counter+=1
        row_dict = {}
        row_dict['img_path'] = path
        row_dict['y'] = y
        rows_list.append(row_dict)
    print((progress/length)*100)
    progress+=1
    clear_output(wait=True)
df = pd.DataFrame(rows_list)
df.to_csv(save_path+'../labels.csv')

In [ ]:
unique = df['y'].value_counts()
print(sum(unique.filter(regex = '^(?!.*-1).*$').values))
print(sum(unique.filter(regex = '^-1$').values))
print(sum(unique.values))